In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/d3_wikihow.csv")

In [3]:
len(df[df.Source == "wikihow"])

15914

In [4]:
df1 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-01.txt.gz", error_bad_lines=False, sep="\t")
df2 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-02.txt.gz", error_bad_lines=False, sep="\t")
df3 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-03.txt.gz", error_bad_lines=False, sep="\t")
df4 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-04.txt.gz", error_bad_lines=False, sep="\t")
df5 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-05.txt.gz", error_bad_lines=False, sep="\t")
df6 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-06.txt.gz", error_bad_lines=False, sep="\t")
df7 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-07.txt.gz", error_bad_lines=False, sep="\t")
df8 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-08.txt.gz", error_bad_lines=False, sep="\t")
df9 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-09.txt.gz", error_bad_lines=False, sep="\t")
df10 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-10.txt.gz", error_bad_lines=False, sep="\t")
aol = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Preprocessing
aol = aol.drop_duplicates(['AnonID', 'Query'], keep='first')
aol = aol[~aol['Query'].str.lower().str.contains('|'.join(["www.", ".com", ".uk", ".us", ".edu"]), na=False)]
aol["QueryTime"] = pd.to_datetime(aol["QueryTime"])
aol = aol.dropna(subset=['Query'])

In [6]:
howto = aol[aol['Query'].str.lower().str.contains("how to", na=False)]

In [11]:
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS = 100000

corpus = howto.Query.tolist() + df.Query.tolist()

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 32630 unique tokens.


In [32]:
MAX_SEQUENCE_LENGTH = max([len(i) for i in corpus])
x = pad_sequences(tokenizer.texts_to_sequences(corpus), maxlen=MAX_SEQUENCE_LENGTH)

In [17]:
from utils import *

max_words = len(word_index)
path = "/Users/jarana/workspace/WikiHow-Task-Based/"
embedding_layer = get_pretrain_embeddings(path, max_words, word_index)


Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.


W0907 17:06:31.219952 4510414272 deprecation_wrapper.py:119] From /Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Token num: 32630, Found Tokens: 28931


In [117]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalMaxPooling1D

model = Sequential([embedding_layer, GlobalMaxPooling1D()])

In [123]:
aol_howto = howto.Query.tolist()
gg_query = df[df.Source != "wikihow"].Query.tolist()

x_howto = pad_sequences(tokenizer.texts_to_sequences(aol_howto), maxlen=MAX_SEQUENCE_LENGTH)
x_gg = pad_sequences(tokenizer.texts_to_sequences(gg_query), maxlen=MAX_SEQUENCE_LENGTH)

In [124]:
x_gg = model.predict(x_gg)

In [125]:
x_aol = model.predict(x_howto)

In [109]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [131]:
x_aol.shape

(56172, 300)

In [152]:
a = [[1,2,3,4], [5,6,7,8],[1,2,3,4], [1,24,4,1]]
b = [[1,2,3,4], [1,24,4,1]]

In [153]:
cosine_similarity(a,b)

array([[1.        , 0.48692212],
       [0.96886393, 0.57544508],
       [1.        , 0.48692212],
       [0.48692212, 1.        ]])

In [156]:
sim_idx = np.argmax(cosine_similarity(x_aol, x_gg), axis=-1)

In [163]:
sim_idx

array([31236, 76365, 92054, ..., 84954, 64113, 91112])

In [165]:
np.save("aol_wiki_gg_sim.npy", sim_idx)

In [161]:
for i in range(1000):
    print(aol_howto[i] +"\t\t"+ gg_query[sim_idx[i]])


how to make tulle butterflies		how to attract butterflies to yourself
how to paint concrete floor		how to lay carpet on concrete garage floor
pageant hair extentions how tos		how to put in clip in hair extensions yourself
how to uninstall a kitchen facuit		how to completely uninstall a program
how to uninstall a kitchen faucet		how to deep clean a kitchen
how to change timing belts		how to change an alternator belt
how to change hyndia timing belts		how to change an alternator belt
how to get free web on nextel phones		how to get free ringtones on iphone
how to make quajada		how to get to runespan
how to relief gas		how to relieve trapped gas
how to make macaroni and cheese		how to make ham and cheese sandwich
how to tie a tie		how to tie a simple tie
how to personalize your locker		how to make your locker awesome
how to have great speed for data entry		learn how to do data entry
how to count from 1-3 in differant laungues		how to make six pack in 1 month
how to count from 1-3 in diffe

In [130]:
for i in range(len(x_aol)):
    sim = [cosine_similarity([x_aol[i]], [j])[0][0] for j in x_gg]
    print(aol_howto[i] +"\t\t"+ gg_query[np.argmax(sim)])
    

how to make tulle butterflies		how to attract butterflies to yourself
how to paint concrete floor		how to lay carpet on concrete garage floor
pageant hair extentions how tos		how to put in clip in hair extensions yourself
how to uninstall a kitchen facuit		how to completely uninstall a program
how to uninstall a kitchen faucet		how to deep clean a kitchen
how to change timing belts		how to change an alternator belt
how to change hyndia timing belts		how to change an alternator belt


KeyboardInterrupt: 